In [1]:
import keras
import pandas as pd
df = pd.read_csv('amazon_cells_labelled.txt', sep='\t', header=None)
df.head()



,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [21]:
from keras.preprocessing.text import Tokenizer


tok = Tokenizer()
print(df[0])

tok.fit_on_texts(df[0])

plug_idx = tok.word_index['plug']
print(f"plug_idx: {plug_idx}")
print("tok.index_word[plug_idx]", tok.index_word[plug_idx])

word_to_index = tok.word_index
print(word_to_index)

0      So there is no way for me to plug it in here i...
1                            Good case, Excellent value.
2                                 Great for the jawbone.
3      Tied to charger for conversations lasting more...
4                                      The mic is great.
                             ...                        
995    The screen does get smudged easily because it ...
996    What a piece of junk.. I lose more calls on th...
997                         Item Does Not Match Picture.
998    The only thing that disappoint me is the infra...
999    You can not answer calls with the unit, never ...
Name: 0, Length: 1000, dtype: object
plug_idx: 155
tok.index_word[plug_idx] plug
{'the': 1, 'i': 2, 'and': 3, 'it': 4, 'is': 5, 'a': 6, 'this': 7, 'to': 8, 'phone': 9, 'my': 10, 'for': 11, 'of': 12, 'not': 13, 'with': 14, 'very': 15, 'great': 16, 'was': 17, 'on': 18, 'in': 19, 'that': 20, 'good': 21, 'have': 22, 'you': 23, 'product': 24, 'quality': 25, 'had': 26, 'headse

In [10]:
seq = tok.texts_to_sequences(df[0])
seq[0]

[33,
 117,
 5,
 53,
 214,
 11,
 47,
 8,
 155,
 4,
 19,
 337,
 19,
 1,
 546,
 416,
 2,
 241,
 190,
 6,
 812]

In [9]:
#  correct import
from keras.utils import pad_sequences

print(' '.join(tok.index_word[i] for i in seq[0]))

# 길이를 맞춰주기
MAXLEN = max(len(s) for s in seq)
pad = pad_sequences(seq, MAXLEN)
pad[0]


so there is no way for me to plug it in here in the us unless i go by a converter


array([  0,   0,   0,   0,   0,   0,   0,   0,   0,  33, 117,   5,  53,
       214,  11,  47,   8, 155,   4,  19, 337,  19,   1, 546, 416,   2,
       241, 190,   6, 812], dtype=int32)

In [12]:
from sklearn.model_selection import train_test_split
# 임베딩 토큰화한 pad와 라벨인 df[1]
X_train, X_test, y_train, y_test = train_test_split(pad, df[1], test_size=.2, random_state=1234)


NUM_WORDS = len(tok.index_word) + 1
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

rnn = Sequential()
#  단어를 밀집 벡터로 매핑하는 데 사용되며, 입력 차원, 출력 차원 및 입력 길이가 설정
rnn.add(Embedding(input_dim=NUM_WORDS, output_dim=8, input_length=MAXLEN, mask_zero=True))
# LSTM 레이어를 모델에 추가
rnn.add(LSTM(16, return_sequences=False))
# 감정 분석을 위한 이진 분류를 수행하며 시그모이드 활성화 함수
rnn.add(Dense(1, activation='sigmoid'))
rnn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 8)             15032     
                                                                 
 lstm (LSTM)                 (None, 16)                1600      
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 16,649
Trainable params: 16,649
Non-trainable params: 0
_________________________________________________________________


In [13]:
from keras.optimizers import Adam
# Adam 옵티마이저: 신경망 모델의 가중치를 업데이트하는 최적화 알고리즘
#  이진 분류 문제를 다루기 때문에 이진 교차 엔트로피 손실 함수
# 모델을 평가할 때 정확도를 측정
rnn.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
# 전체 학습 데이터를 10번 반복하여 모델을 학습한다는 것을 의미
rnn.fit(X_train, y_train, epochs=10)

Epoch 1/10


2023-10-04 21:25:34.098583: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


25/25 [==============================] - 1s 5ms/step - loss: 0.6928 - accuracy: 0.5337
Epoch 2/10
25/25 [==============================] - 0s 4ms/step - loss: 0.6888 - accuracy: 0.7050
Epoch 3/10
25/25 [==============================] - 0s 4ms/step - loss: 0.6691 - accuracy: 0.8450
Epoch 4/10
25/25 [==============================] - 0s 4ms/step - loss: 0.5707 - accuracy: 0.8737
Epoch 5/10
25/25 [==============================] - 0s 4ms/step - loss: 0.4279 - accuracy: 0.9013
Epoch 6/10
25/25 [==============================] - 0s 4ms/step - loss: 0.3043 - accuracy: 0.9488
Epoch 7/10
25/25 [==============================] - 0s 4ms/step - loss: 0.2426 - accuracy: 0.9575
Epoch 8/10
25/25 [==============================] - 0s 4ms/step - loss: 0.1975 - accuracy: 0.9712
Epoch 9/10
25/25 [==============================] - 0s 4ms/step - loss: 0.1719 - accuracy: 0.9762
Epoch 10/10
25/25 [==============================] - 0s 4ms/step - loss: 0.1314 - accuracy: 0.9825


In [14]:
y_rnn = (rnn.predict(X_test) > 0.5)*1

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_rnn)

7/7 [==============================] - 0s 2ms/step


0.815

In [15]:
# - 역방향 
# 단어처리 순서를 앞에서 뒤로 하는 것이 아니라 역방향으로 뒤에서 앞으로도 할 수 있다. 순환신경망 레이어에 go_backwards=True를 추가해주면 된다.
rnn_goback = Sequential()
rnn_goback.add(Embedding(input_dim=NUM_WORDS, output_dim=8, input_length=MAXLEN, mask_zero=True))
rnn_goback.add(LSTM(16, return_sequences=False, go_backwards=True))
rnn_goback.add(Dense(1, activation='sigmoid'))

rnn_goback.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
rnn_goback.fit(X_train, y_train, epochs=10)




Epoch 1/10
25/25 [==============================] - 1s 4ms/step - loss: 0.6927 - accuracy: 0.5113
Epoch 2/10
25/25 [==============================] - 0s 4ms/step - loss: 0.6874 - accuracy: 0.5750
Epoch 3/10
25/25 [==============================] - 0s 4ms/step - loss: 0.6655 - accuracy: 0.8012
Epoch 4/10
25/25 [==============================] - 0s 4ms/step - loss: 0.5722 - accuracy: 0.8712
Epoch 5/10
25/25 [==============================] - 0s 4ms/step - loss: 0.4299 - accuracy: 0.9050
Epoch 6/10
25/25 [==============================] - 0s 6ms/step - loss: 0.3347 - accuracy: 0.9400
Epoch 7/10
25/25 [==============================] - 0s 4ms/step - loss: 0.2534 - accuracy: 0.9625
Epoch 8/10
25/25 [==============================] - 0s 4ms/step - loss: 0.2085 - accuracy: 0.9688
Epoch 9/10
25/25 [==============================] - 0s 4ms/step - loss: 0.1682 - accuracy: 0.9800
Epoch 10/10
25/25 [==============================] - 0s 4ms/step - loss: 0.1352 - accuracy: 0.9862


In [16]:
y_rnn_goback = (rnn_goback.predict(X_test) > 0.5)*1

from sklearn.metrics import accuracy_score
accuracy_score(y_rnn_goback, y_rnn)

7/7 [==============================] - 0s 2ms/step


0.74

In [17]:
 # - 양방향
# 순방향 RNN과 양방향 RNN을 합치면 양방향 RNN이 된다. Bidirectional을 사용한다.
from keras.layers import Bidirectional
rnn_bidirec = Sequential()
rnn_bidirec.add(Embedding(input_dim=NUM_WORDS, output_dim=8, input_length=MAXLEN, mask_zero=True))
rnn_bidirec.add(Bidirectional(LSTM(16, return_sequences=False)))
rnn_bidirec.add(Dense(1, activation='sigmoid'))


rnn_bidirec.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
rnn_bidirec.fit(X_train, y_train, epochs=10)




Epoch 1/10
25/25 [==============================] - 2s 5ms/step - loss: 0.6926 - accuracy: 0.5325
Epoch 2/10
25/25 [==============================] - 0s 4ms/step - loss: 0.6875 - accuracy: 0.7788
Epoch 3/10
25/25 [==============================] - 0s 4ms/step - loss: 0.6647 - accuracy: 0.8487
Epoch 4/10
25/25 [==============================] - 0s 4ms/step - loss: 0.5274 - accuracy: 0.8838
Epoch 5/10
25/25 [==============================] - 0s 4ms/step - loss: 0.3299 - accuracy: 0.9237
Epoch 6/10
25/25 [==============================] - 0s 4ms/step - loss: 0.2229 - accuracy: 0.9638
Epoch 7/10
25/25 [==============================] - 0s 4ms/step - loss: 0.1636 - accuracy: 0.9762
Epoch 8/10
25/25 [==============================] - 0s 4ms/step - loss: 0.1317 - accuracy: 0.9787
Epoch 9/10
25/25 [==============================] - 0s 4ms/step - loss: 0.1061 - accuracy: 0.9862
Epoch 10/10
25/25 [==============================] - 0s 4ms/step - loss: 0.1041 - accuracy: 0.9837


In [18]:
y_rnn_bidirec = (rnn_bidirec.predict(X_test) > 0.5)*1

from sklearn.metrics import accuracy_score
accuracy_score(y_rnn_bidirec, y_rnn)


7/7 [==============================] - 1s 2ms/step


0.85

In [19]:
# CNN 모델 추가

from keras.layers import Conv1D, GlobalMaxPooling1D


cnn = Sequential()
cnn.add(Embedding(input_dim=NUM_WORDS, output_dim=8, input_length=MAXLEN))
cnn.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
cnn.add(GlobalMaxPooling1D())
cnn.add(Dense(1, activation='sigmoid'))

cnn.summary()

# 모델 컴파일 및 학습
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn.fit(X_train, y_train, epochs=10)

# 테스트 데이터 평가
accuracy = cnn.evaluate(X_test, y_test)[1]
print("Test Accuracy:", accuracy)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 30, 8)             15032     
                                                                 
 conv1d (Conv1D)             (None, 28, 16)            400       
                                                                 
 global_max_pooling1d (Globa  (None, 16)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 15,449
Trainable params: 15,449
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
25/25 [==============================] - 0s 743us/step - loss: 0.6920 - accuracy: 0.5138
Epoch 2/10
25/25

In [20]:
# 앙상블 모델 = LSTM 모델 + CNN 모델

from keras.models import Model
from keras.layers import Input, concatenate


lstm_input = Input(shape=(MAXLEN,))
lstm_embedding = Embedding(input_dim=NUM_WORDS, output_dim=8, input_length=MAXLEN)(lstm_input)
lstm_output = LSTM(16, return_sequences=False)(lstm_embedding)

# CNN 모델
cnn_input = Input(shape=(MAXLEN,))
cnn_embedding = Embedding(input_dim=NUM_WORDS, output_dim=8, input_length=MAXLEN)(cnn_input)
cnn_conv = Conv1D(filters=16, kernel_size=3, activation='relu')(cnn_embedding)
cnn_pool = GlobalMaxPooling1D()(cnn_conv)

# LSTM과 CNN 모델 병합
merged = concatenate([lstm_output, cnn_pool])
output = Dense(1, activation='sigmoid')(merged)

# 앙상블 모델 생성
ensemble_model = Model(inputs=[lstm_input, cnn_input], outputs=output)
ensemble_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

ensemble_model.summary()

# 학습 및 평가
ensemble_model.fit([X_train, X_train], y_train, epochs=10)
accuracy = ensemble_model.evaluate([X_test, X_test], y_test)[1]
print("Test Accuracy:", accuracy)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 embedding_5 (Embedding)        (None, 30, 8)        15032       ['input_2[0][0]']                
                                                                                                  
 embedding_4 (Embedding)        (None, 30, 8)        15032       ['input_1[0][0]']                
                                                                                              